In [1]:
# Import dependencies
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import time

In [2]:
# Scrape website
# html=browser.html

# Find all the car listings on the page
# car_list=car_soup.find_all('div', class_='vehicle-card')

# Testing scraping for the first listing on the site
# car_model=car_soup.find('h2', class_='title')
# car_status=car_soup.find('p', class_='stock-type')
# car_price=car_soup.find('span', class_='primary-price')
# car_dealer=car_soup.find('div', class_='dealer-name')
# dealer_city=car_soup.find('div', class_='miles-from')

# print(car_model, car_price, car_dealer, car_status, dealer_city)


In [3]:
# Testing to make sure text that is stripped is correct
#model=car_model.get_text(strip=True)
#status=car_status.get_text(strip=True)
#price=car_price.get_text(strip=True)
#dealer=car_dealer.get_text(strip=True)
#city=dealer_city.get_text(strip=True)

#print(model, status, price, dealer, city)

In [4]:
# Set up Splinter
browser = Browser('chrome')

# Base URL and initial page
base_url = 'https://www.cars.com/shopping/results/?dealer_id=&include_shippable=true&keyword=&list_price_max=&list_price_min=&makes[]=toyota&maximum_distance=150&mileage_max=&models[]=toyota-rav4&models[]=toyota-rav4_hybrid&monthly_payment=&page={}&page_size=100&sort=best_match_desc&stock_type=all&year_max=&year_min=&zip=61606'
page_number = 1
url = base_url.format(page_number)

# Create lists to store details
models = []
statuses = []
mileages = []
prices = []
dealers = []
cities = []
urls = []  # List to store URLs

# Set to store unique identifiers of scraped cars
scraped_car_ids = set()

# Function to scrape current page
def scrape_page(url):
    browser.visit(url)
    time.sleep(5)  # Adjust as necessary
    
    html = browser.html
    car_soup = soup(html, 'html.parser')
    car_list = car_soup.find_all('div', class_='vehicle-card')

    if not car_list:
        print("No more car listings found. Exiting.")
        return False

    for car in car_list:
        # Extract car model
        car_model = car.find('h2', class_='title')
        model = str(car_model.get_text(strip=True)) if car_model else 'N/A'

        # Extract car mileage
        car_mileage = car.find('div', class_='mileage')
        mileage = str(car_mileage.get_text(strip=True)) if car_mileage else 'N/A'

        # Extract car status
        car_status = car.find('p', class_='stock-type')
        status = str(car_status.get_text(strip=True)) if car_status else 'N/A'

        # Extract car price
        car_price = car.find('span', class_='primary-price')
        price = str(car_price.get_text(strip=True)) if car_price else 'N/A'

        # Extract dealer name
        car_dealer_div = car.find('div', class_='dealer-name')
        if car_dealer_div:
            car_dealer = car_dealer_div.find('strong')
            dealer = str(car_dealer.get_text(strip=True)) if car_dealer else 'N/A'
        else:
            dealer = 'N/A'

        # Extract dealer city
        dealer_city = car.find('div', class_='miles-from')
        city = str(dealer_city.get_text(strip=True)) if dealer_city else 'N/A'

        # Extract car URL
        car_url = car.find('a', class_='vehicle-card-link')['href'] if car.find('a', class_='vehicle-card-link') else 'N/A'

        # Check if this car has already been scraped
        if car_url in scraped_car_ids:
            continue  # Skip if already scraped
        
        # Add car URL to set of scraped car IDs
        scraped_car_ids.add(car_url)

        # Append details to lists
        models.append(model)
        statuses.append(status)
        prices.append(price)
        dealers.append(dealer)
        cities.append(city)
        mileages.append(mileage)
        urls.append(car_url)  # Append URL to list

    return True

# Counter to track consecutive pages with no new data
no_new_data_count = 0
max_no_new_data_allowed = 3  # Adjust as necessary

# Timeout settings
timeout_seconds = 180  # 4 minutes timeout
start_time = time.time()

# Loop through pages
while True:
    if time.time() - start_time > timeout_seconds:
        print(f"Timeout ({timeout_seconds} seconds) exceeded. Exiting.")
        break
    
    if not scrape_page(url):
        # Increment no_new_data_count if no new data found
        no_new_data_count += 1
        if no_new_data_count >= max_no_new_data_allowed:
            print(f"No new data found for {max_no_new_data_allowed} consecutive pages. Exiting.")
            break
    else:
        # Reset no_new_data_count if new data found
        no_new_data_count = 0
    
    # Increment page number and update URL
    page_number += 1
    url = base_url.format(page_number)
    print(f"Scraping page {page_number}...")

# Print number of cars scraped
print(f"Number of unique cars scraped: {len(scraped_car_ids)}")

Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
Scraping page 18...
Scraping page 19...
Scraping page 20...
Scraping page 21...
Scraping page 22...
Scraping page 23...
Scraping page 24...
Scraping page 25...
Scraping page 26...
Timeout (180 seconds) exceeded. Exiting.
Number of unique cars scraped: 1204


In [5]:
# Create df
data={
    'Model': models,
    'Mileage (mi)': mileages,
    'Status': statuses,
    'Price (USD)': prices,
    'Dealer': dealers,
    'City': cities ,
    'URL': urls
    }
car_df= pd.DataFrame(data)

In [6]:
car_df

,Model,Mileage (mi),Status,Price (USD),Dealer,City,URL
0,2017 Toyota RAV4 XLE,"76,200 mi.",Used,"$19,700",The Autobarn Volkswagen of Countryside,"Countryside, IL",/vehicledetail/a678734e-1445-4b00-b9d3-25fb078...
1,2009 Toyota RAV4 Base,"147,000 mi.",Used,"$7,550",N/A,"Dunlap, IL (11 mi.)",/vehicledetail/0c870d0c-43b2-4b44-8ffb-d69fc7d...
2,2024 Toyota RAV4 Hybrid Woodland Edition,"2,178 mi.",Used,"$36,950",Sam Leman Toyota - Bloomington,"Bloomington, IL (38 mi.)",/vehicledetail/85eb36b7-a96c-4617-9986-9877ef0...
3,2015 Toyota RAV4 Limited,"118,204 mi.",Used,"$17,450",Sam Leman Toyota - Bloomington,"Bloomington, IL (38 mi.)",/vehicledetail/5006dd39-b2b6-4bb5-99c5-767f842...
4,2018 Toyota RAV4 Hybrid XLE,"38,079 mi.",Used,"$24,998",CarMax Newark - Offering Express Pickup and Ho...,N/A,/vehicledetail/d3da6be5-bb98-4458-96da-4c4e3e1...
...,...,...,...,...,...,...,...
1199,2023 Toyota RAV4 XLE,"40,082 mi.",Used,"$28,992",Carver Toyota of Columbus,"Taylorsville, IN (217 mi.)",/vehicledetail/a56320f9-00d7-45c6-af61-7325d54...
1200,2023 Toyota RAV4 LE,"18,212 mi.",Used,"$27,538",Moses Honda,"Huntington, WV (417 mi.)",/vehicledetail/b0c947cd-918e-440c-8805-ebea13e...
1201,2022 Toyota RAV4 Hybrid XLE,"45,397 mi.",Used,"$32,950",Lexus of Chattanooga,"Chattanooga, TN (459 mi.)",/vehicledetail/9bf36770-3988-45df-9a54-116f76f...
1202,2021 Toyota RAV4 Hybrid Limited,"24,854 mi.",Used,"$35,180",Great Lakes Subaru,"Findlay, OH (315 mi.)",/vehicledetail/afc8476f-3fc5-4677-9127-f02c6e7...


In [5]:
# Checking to make sure df is formatted correctly
# print(car_df.to_string())

In [7]:
# Close the browser
browser.quit()

In [8]:
# Display DF
print("Original DataFrame:")
print(car_df)
print()

Original DataFrame:
                                         Model Mileage (mi) Status  \
0                         2017 Toyota RAV4 XLE   76,200 mi.   Used   
1                        2009 Toyota RAV4 Base  147,000 mi.   Used   
2     2024 Toyota RAV4 Hybrid Woodland Edition    2,178 mi.   Used   
3                     2015 Toyota RAV4 Limited  118,204 mi.   Used   
4                  2018 Toyota RAV4 Hybrid XLE   38,079 mi.   Used   
...                                        ...          ...    ...   
1199                      2023 Toyota RAV4 XLE   40,082 mi.   Used   
1200                       2023 Toyota RAV4 LE   18,212 mi.   Used   
1201               2022 Toyota RAV4 Hybrid XLE   45,397 mi.   Used   
1202           2021 Toyota RAV4 Hybrid Limited   24,854 mi.   Used   
1203                2019 Toyota RAV4 Hybrid LE   75,502 mi.   Used   

     Price (USD)                                             Dealer  \
0        $19,700             The Autobarn Volkswagen of Countryside 

In [9]:
# Check for missing values:
missing_values= car_df.isnull().sum()
print("Missing values:")
print(missing_values)
print()

Missing values:
Model           0
Mileage (mi)    0
Status          0
Price (USD)     0
Dealer          0
City            0
URL             0
dtype: int64



In [10]:
# Split 'Model' column into 'Year', 'Manufacturer', 'Model', and 'Trim'
# Handling 'CR-V' and 'CR-V Hybrid' correctly
split_df = car_df['Model'].str.extract(r'(?P<Year>\d{4})\s+(?P<Manufacturer>[\w-]+)\s+(?P<Model>RAV4(?:\s+Hybrid)?)\s+(?P<Trim>.*)')
#  Captures group called 'Year' (?P<Year> ; \d{4} matches to 4 digits
# Matches 1+ whitespace characters to account for space between Year and Manufacturer \s+
# Captures another group called Manufacturer, matches by 1+ words (?P<Manufacturer>[\w-]+)
# Captures group called Model (?P<Model> ; CR-V(?:\s+Hybrid)?) Matches to string "CR-V" 
# or by a white space (?:\s) plus the string "Hybrid" +Hybrid)?)
# Captures group called Trim (?P<Trim>) ; Matches any character 0+ times 
# (i.e. captures everything after Model as the Trim) using .*

# Replace 'Not Priced' with NaN and convert 'Price' column to numeric
car_df['Price (USD)'] = pd.to_numeric(car_df['Price (USD)'].replace('Not Priced', pd.NA).str.replace('[\$,]', '', regex=True), errors='coerce')

# Clean the 'Mileage' column
car_df['Mileage (mi)'] = pd.to_numeric(car_df['Mileage (mi)'].str.replace('mi.', '').str.replace(',', '').str.strip(), errors='coerce')

# Concatenate split_df with the remaining columns from car_df
car_df = pd.concat([split_df, car_df[['Status', 'Price (USD)', 'Dealer', 'City', 'Mileage (mi)', 'URL']]], axis=1)

# Reorder columns as required
car_df = car_df[['Year', 'Manufacturer', 'Model', 'Trim', 'Mileage (mi)', 'Status', 'Price (USD)', 'Dealer', 'City', 'URL']]

# Display the updated DataFrame
print("Final DataFrame:")
print(car_df)

Final DataFrame:
      Year Manufacturer        Model              Trim  Mileage (mi) Status  \
0     2017       Toyota         RAV4               XLE       76200.0   Used   
1     2009       Toyota         RAV4              Base      147000.0   Used   
2     2024       Toyota  RAV4 Hybrid  Woodland Edition        2178.0   Used   
3     2015       Toyota         RAV4           Limited      118204.0   Used   
4     2018       Toyota  RAV4 Hybrid               XLE       38079.0   Used   
...    ...          ...          ...               ...           ...    ...   
1199  2023       Toyota         RAV4               XLE       40082.0   Used   
1200  2023       Toyota         RAV4                LE       18212.0   Used   
1201  2022       Toyota  RAV4 Hybrid               XLE       45397.0   Used   
1202  2021       Toyota  RAV4 Hybrid           Limited       24854.0   Used   
1203  2019       Toyota  RAV4 Hybrid                LE       75502.0   Used   

      Price (USD)                 

C:\Users\awsom\AppData\Local\Temp\ipykernel_19740\1332999187.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  car_df['Mileage (mi)'] = pd.to_numeric(car_df['Mileage (mi)'].str.replace('mi.', '').str.replace(',', '').str.strip(), errors='coerce')


In [11]:
car_df

,Year,Manufacturer,Model,Trim,Mileage (mi),Status,Price (USD),Dealer,City,URL
0,2017,Toyota,RAV4,XLE,76200.0,Used,19700.0,The Autobarn Volkswagen of Countryside,"Countryside, IL",/vehicledetail/a678734e-1445-4b00-b9d3-25fb078...
1,2009,Toyota,RAV4,Base,147000.0,Used,7550.0,N/A,"Dunlap, IL (11 mi.)",/vehicledetail/0c870d0c-43b2-4b44-8ffb-d69fc7d...
2,2024,Toyota,RAV4 Hybrid,Woodland Edition,2178.0,Used,36950.0,Sam Leman Toyota - Bloomington,"Bloomington, IL (38 mi.)",/vehicledetail/85eb36b7-a96c-4617-9986-9877ef0...
3,2015,Toyota,RAV4,Limited,118204.0,Used,17450.0,Sam Leman Toyota - Bloomington,"Bloomington, IL (38 mi.)",/vehicledetail/5006dd39-b2b6-4bb5-99c5-767f842...
4,2018,Toyota,RAV4 Hybrid,XLE,38079.0,Used,24998.0,CarMax Newark - Offering Express Pickup and Ho...,N/A,/vehicledetail/d3da6be5-bb98-4458-96da-4c4e3e1...
...,...,...,...,...,...,...,...,...,...,...
1199,2023,Toyota,RAV4,XLE,40082.0,Used,28992.0,Carver Toyota of Columbus,"Taylorsville, IN (217 mi.)",/vehicledetail/a56320f9-00d7-45c6-af61-7325d54...
1200,2023,Toyota,RAV4,LE,18212.0,Used,27538.0,Moses Honda,"Huntington, WV (417 mi.)",/vehicledetail/b0c947cd-918e-440c-8805-ebea13e...
1201,2022,Toyota,RAV4 Hybrid,XLE,45397.0,Used,32950.0,Lexus of Chattanooga,"Chattanooga, TN (459 mi.)",/vehicledetail/9bf36770-3988-45df-9a54-116f76f...
1202,2021,Toyota,RAV4 Hybrid,Limited,24854.0,Used,35180.0,Great Lakes Subaru,"Findlay, OH (315 mi.)",/vehicledetail/afc8476f-3fc5-4677-9127-f02c6e7...


In [10]:
#print(car_df.to_string())

In [12]:
car_df.to_excel("RAV4.xlsx")